In [1]:
import torch
import torch.nn as nn
from torch.nn import Linear, Parameter
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, MessagePassing
from torch_geometric.transforms import ToSparseTensor
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.datasets import TUDataset, Planetoid
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

In [ ]:
class CustomGraphDataset(torch.utils.data.Dataset):
    def __init__(self, mutag, protein):
        self.mutag = TUDataset(root='/tmp/MUTAG', name='MUTAG') if mutag else None
        self.protein = Planetoid(root='/tmp/Protein', name='Protein') if protein else None

    def __len__(self):
        return len(self.mutag) if self.mutag is not None else len(self.protein)

    def __getitem__(self, idx):
        if self.mutag is not None:
            return self.mutag[idx]
        elif self.protein is not None:
            return self.protein[idx]


# To use the data loader, split the data into training and validation sets,
# and create data loaders for the two datasets separately.

dataset = CustomGraphDataset(mutag=True, protein=True)

# Perform a train/test split
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class EGConv(MessagePassing):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim, num_heads):
        super(EGConv, self).__init__(aggr='add')  # "Add" aggregation (Step 5).

        # EdgeCNN for processing edge features
        self.edge_cnn = nn.Sequential(
            Linear(edge_feature_dim, hidden_dim),
            nn.ReLU(),
            Linear(hidden_dim, hidden_dim)
        )

        # GAT layer for processing node and edge features
        self.gat = GATConv(
            in_channels=node_feature_dim + hidden_dim,
            out_channels=hidden_dim,
            heads=num_heads,
            concat=True
        )

    def forward(self, x, edge_index, edge_attr):
        # EdgeCNN processing is done dynamically in the message function.
        # Apply GAT layer afterwards.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.gat(x, edge_index)

    def message(self, x_j, edge_index_i, edge_attr):
        # Apply EdgeCNN in the message function
        edge_attr = self.edge_cnn(edge_attr)

        # Concatenate edge and node features
        x_j = torch.cat([x_j, edge_attr], dim=1)

        return x_j

In [ ]:
# Instantiate the model
node_feature_dim = 64
edge_feature_dim = 16
hidden_dim = 128
num_heads = 4
model = EdgeGAT(node_feature_dim, edge_feature_dim, hidden_dim, num_heads)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create a DataLoader for your dataset
dataset = CustomGraphDataset()
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

    # Print or log the training loss at the end of each epoch
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Evaluation (you may need a separate validation dataset)
model.eval()